In [ ]:
# Import geemap for mapping
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee
import datetime

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import os
from glob import glob
from rich.progress import track
    

In [40]:
import zipfile

In [2]:
# Adds Earth Engine dataset
ulx = 69
uly = 20.37
llx = 74
lly = 15.7
polygon = ee.Geometry.Rectangle([ulx, lly, llx, uly])
# ee.Geometry.Polygon([[69, 20.37], [74, 20.37], [74, 15.7], [69, 15.7], [69, 20.37]])
collection = (ee.ImageCollection('HYCOM/sea_water_velocity').filterBounds(polygon).filterDate(datetime.datetime(2020, 11, 5), datetime.datetime(2020, 11, 15)))

In [3]:
image_ids = [x['id'] for x in collection.select('velocity_u_0').getInfo()['features']]

In [5]:
# functions to calculate speed and magintude raster bands from the input image collection using GEE methods.
def getMagnitude(image, polygon):
    """ gives the speed magnitude of the velocity vector as m/s """
    uzero = image.select('velocity_u_0')
    vzero = image.select('velocity_v_0')
    mag = (uzero.multiply(uzero).add(vzero.multiply(vzero))).sqrt().rename('speed_0').toInt16()
    
    return image.addBands([mag])

def getDirection(image, polygon):
    """ gives the angle of velocity direction with respect to true north in the clockwise direction (0-360) """
    uzero = image.select('velocity_u_0')
    vzero = image.select('velocity_v_0')
    angle = (uzero.atan2(vzero).multiply(57.29577)).rename('direction_0')
    angle = angle.where(angle.lt(0), angle.add(360))
    angle = angle.where(angle.gt(90), angle.subtract(450).multiply(-1))
    angle = angle.where(angle.lte(90), angle.subtract(90).multiply(-1)).toInt16()
    
    return image.addBands([angle])

def combineRaster(image):
    """ combines the bands of u, v velocity components with the newly calculated speed magnitude and direction in a separate raster"""
    uzero = ee.Image(image.select('velocity_u_0'))
    vzero = ee.Image(image.select('velocity_v_0'))
    mag = ee.Image(image.select('speed_0'))
    angle = ee.Image(image.select('direction_0'))
    combined = ee.Image.cat([uzero, vzero, mag, angle])
    
    return combined
    

In [81]:
def download_url(url, resol, default_path='../DataOutput/'):
    """ Iterate through file chunks and download the file to local. """
    
    cd = os.path.join(default_path, resol)
    if not os.path.exists(cd):
        os.makedirs(cd)

    r = requests.get(url, stream=True)
    filename = r.headers['Content-disposition'].split('=')[-1]
    path = os.path.join(cd, filename)

    with open(path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                f.flush()
    return path

In [73]:
def main(img):
    """ Fetch the images by id and run the calculations, before exporting the combined raster to both Drive and Local."""
    
    image = ee.Image(img)
    image = getMagnitude(image, polygon)
    image = getDirection(image, polygon)
    image = image.clip(polygon)
    output = combineRaster(image)
    
    date = img.split('/')[-1]
    # download data for different resolutions by scaling
    scale = {'0.08deg':'', '5km':{'scale':5000}, '10km':{'scale':10000}}
    for key, val in scale.items():
        url = output.getDownloadURL() if key == '0.08deg' else output.getDownloadURL(params=val)
        path = download_url(url, key)

        # images will be exported to the Drive as GeoTiff to the default My Drive folder with `task_name`
#     task_name = f"hycom_swv_{date}"
#     task = ee.batch.Export.image.toDrive(output, task_name)
#     task.start()
    return output

output_collection = [main(img) for img in track(image_ids, description='Calculating Speed and Direction for Raster ROI.')]

Output()

In [74]:
# Unzip and delete zip file outputs and combine speed and direction rasters separately
extension = '.zip'
files = glob('../DataOutput/*/*.zip')
for item in track(files, description='Unzipping files:'):
    if item.endswith(extension):
        zip_ref = zipfile.ZipFile(item) 
        zip_ref.extractall(os.path.dirname(item))
        zip_ref.close()
        os.remove(item)

Output()

## Visualization

### Creates an interactive map of Raw data

In [75]:
Map1 = emap.Map(center=[18,72], zoom=3)

In [76]:
# Sets visualization parameters for base data.
vis_params = {
    'min': -1000,
    'max': 4000,
    'bands' : ['velocity_u_0', 'velocity_v_0', 'velocity_v_0']
}

# Adds Earth Engine layers to Map
Map1.addLayer(collection, vis_params, 'Ocean Currents', True, 1)
Map1.addLayerControl()
Map1

Map(center=[18, 72], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=Fa…

Element.toDictionary: Parameter 'element' is required.


In [77]:
out_collection = ee.ImageCollection.fromImages(output_collection)

In [78]:
directionImg = out_collection.select('direction_0') # all direction raster bands from 2020-11-05 to 2020-11-14
speedImg = out_collection.select('speed_0') # all speed raster bands from 2020-11-05 to 2020-11-14

### Creates an interactive map of Direction rasters

In [79]:
Map2 = emap.Map(center=[18,72], zoom=7)

In [80]:
# Color Palette RdYlGn : green - min, red - max
Map2.addLayer(directionImg, 
              {
                  'min': 0, 
                   'max':360, 
                   'palette': ["a50026","d73027","f46d43","fdae61","fee08b","d9ef8b","a6d96a","66bd63","1a9850","006837"]
              }, 
              'Direction', True, 1)

# Display the Map
Map2.addLayerControl()
Map2

Map(center=[18, 72], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=Fa…

### Creates an interactive map of Speed rasters

In [61]:
Map3 = emap.Map(center=[18,72], zoom=7)

In [63]:
# Color Palette RdYlGn : green - min, red - max
Map3.addLayer(speedImg, 
              {
                  'min': -100, 
                   'max': 1000, 
                   'palette': ["a50026","d73027","f46d43","fdae61","fee08b","d9ef8b","a6d96a","66bd63","1a9850","006837"]
              }, 
              'Speed', True, 1)

# Display the Map
Map3.addLayerControl()
Map3

Map(center=[18, 72], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=Fa…